# Web3 Exploration

Explore Ethereum blockchain data using Web3.py and Infura.

In [1]:
import sys
from pathlib import Path

# Add project root to Python path
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from utils import get_infura_web3

## Connect to Ethereum

Create an HTTP connection to Infura and fetch the latest block:

In [2]:
# Connect to Infura
w3 = get_infura_web3()

print(f"Connected: {w3.is_connected()}")
print(f"Chain ID: {w3.eth.chain_id}")

Connected: True
Chain ID: 1


In [3]:
# Get the latest block
latest_block = w3.eth.get_block('latest')

print(f"Latest Block Number: {latest_block['number']}")
print(f"Block Hash: {latest_block['hash'].hex()}")
print(f"Timestamp: {latest_block['timestamp']}")
print(f"Gas Used: {latest_block['gasUsed']:,}")
print(f"Gas Limit: {latest_block['gasLimit']:,}")
print(f"Transaction Count: {len(latest_block['transactions'])}")

Latest Block Number: 23928735
Block Hash: b455be9dcbc3deee154edef55daeb1e87c5cd0b4b99048930213b1b12d0768f2
Timestamp: 1764714251
Gas Used: 33,941,760
Gas Limit: 60,000,000
Transaction Count: 80


## Gas Price Information

In [4]:
# Get current gas price
gas_price_wei = w3.eth.gas_price
gas_price_gwei = w3.from_wei(gas_price_wei, 'gwei')

print(f"Current Gas Price: {gas_price_gwei:.2f} Gwei")

Current Gas Price: 0.03 Gwei


In [5]:
block = w3.eth.get_block(23928359 - 1)
transactions = block['transactions']
transaction = transactions[0]
transaction

HexBytes('0x45545d18761620cc1538e83520155426e815f65e5887757c569229a308c87bea')

In [6]:
transaction_dict = w3.eth.get_transaction(transaction)

In [7]:
address = "0x6CA298D2983aB03Aa1dA7679389D955A4eFEE15C"
address.lower() == transaction_dict['to'].lower()

False

In [8]:
import time
import random

def retry_with_backoff(func, max_retries=5, base_delay=0.5, max_delay=30):
    """
    Retry a function with exponential backoff.
    
    Args:
        func: Callable to execute
        max_retries: Maximum number of retry attempts
        base_delay: Initial delay in seconds
        max_delay: Maximum delay between retries
        
    Returns:
        Result of the function call
        
    Raises:
        Exception: If all retries are exhausted
    """
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            # Exponential backoff with jitter
            delay = min(base_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
            time.sleep(delay)


def transactions_with_address(block_number, address):
    """Get all transactions in a block that involve a specific address."""
    block = retry_with_backoff(lambda: w3.eth.get_block(block_number))
    txs = []
    transactions = block['transactions']
    for transaction in transactions:
        transaction_dict = retry_with_backoff(lambda tx=transaction: w3.eth.get_transaction(tx))
        to_addr = transaction_dict.get('to')
        if to_addr and address.lower() == to_addr.lower():
            txs.append(transaction)
    return txs

In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def transactions_with_address_parallel(block_numbers, address, max_workers=10):
    """
    Process multiple blocks in parallel to find transactions involving an address.
    
    Args:
        block_numbers: List of block numbers to process
        address: The address to search for
        max_workers: Number of parallel threads (default 10)
        
    Returns:
        List of lists, where each inner list contains transactions for that block
    """
    results = [None] * len(block_numbers)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and track which block each future corresponds to
        future_to_idx = {
            executor.submit(transactions_with_address, block_num, address): idx
            for idx, block_num in enumerate(block_numbers)
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            results[idx] = future.result()
    
    return results


# Example: Process blocks 23928350 to 23928359 in parallel
address = "0x6CA298D2983aB03Aa1dA7679389D955A4eFEE15C"
block_numbers = list(range(23928350, 23928360))

start = time.time()
results = transactions_with_address_parallel(block_numbers, address, max_workers=10)
end = time.time()

print(f"Processed {len(block_numbers)} blocks in {end - start:.2f} seconds")
print(f"Results: {results}")

Processed 10 blocks in 18.76 seconds
Results: [[], [], [], [], [], [], [], [], [], []]
